In [14]:
import scipy.signal as signal
import numpy as np
import scipy.io.wavfile as wav
import sounddevice as sd
import librosa
from pathlib import Path
import pandas

#c:/Users/egorv/Desktop/BProj
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "code" / "experiments" / "data" / "ASVSpoof2019"
TARGET_SEC = 4.0
TARGET_LEN = int(16000 * TARGET_SEC)

In [15]:
def normalize_duration(y):
    cur_len = len(y)

    if cur_len > TARGET_LEN:
        start = np.random.randint(0, cur_len - TARGET_LEN)
        return y[start:start+TARGET_LEN]
    
    if cur_len < TARGET_LEN:
        pad = TARGET_LEN - cur_len
        return np.pad(y, (0, pad), mode='constant')
    
    else:
        return y
    
class ASVSpoofDataset:
    
    def __init__(self, flac_dir):
        self.files = sorted(Path(flac_dir).glob("*.flac"))

    def __len___(self):
        return len(self.files)
    
    def __getitem__(self,idx):
        y, sr = librosa.load(self.files[idx], sr=None)
        return y, sr

In [16]:
train_flac_dir = (
    DATA_DIR
    / "LA"
    / "ASVSpoof2019_LA_train"
    / "flac"
)

flac_files = ASVSpoofDataset(train_flac_dir)
# y, sr = dataset[0]



In [ ]:
labels_file = DATA_DIR / "LA" / "ASVspoof2019_LA_cm_protocols" / "ASVspoof2019.LA.cm.train.trn.txt"
labels_df = pandas.read_csv(labels_file, sep=r"\s+", header=None)
labels_df = labels_df.rename(columns={0:"speaker",1:"filename",2:"unused",3:"type of attack",4:"target"}).drop(['unused'], axis=1)
dataset = dict(zip(labels_df['filename'], labels_df['target']))